In [39]:
import torch
from torch import nn
from tests_backpropagation import main_test

torch.manual_seed(123)
torch.set_default_dtype(torch.double)

## Class ``MyNet``

Read carefully how ``MyNet`` is implemented in the cell below. In particular:  
- ``n_hid`` is a list of integer, representing the number of hidden units in each hidden layer.   
-  ``MyNet([2, 3, 2]) = MiniNet()`` where ``MiniNet`` is the neural network defined in the fourth tutorial, in which notations are also clarified.     
- ``model.L`` is the number of hidden layers, ``L``   
- ``model.f[l]`` is the activation function of layer ``l``, $f^{[l]}$ (here ``torch.tanh``)   
- ``model.df[l]`` is the derivative of the activation function, $f'^{[l]}$   
- ``model.a[l]``  is the tensor $A^{[l]}$, (shape: ``(1, n(l))``)   
- ``model.z[l]``  is the tensor $Z^{[l]}$, (shape: ``(1, n(l))``)  
- Weights $W^{[l]}$ (shape: ``(n(l+1), n(l))``) and biases $\mathbf{b}^{[l]}$ (shape: ``(n(l+1))``) can be accessed as follows:
```
weights = model.fc[str(l)].weight.data
bias = model.fc[str(l)].bias.data
```

In [40]:
class MyNet(nn.Module):
    def __init__(self, n_l = [2, 3, 2]):
        super().__init__() 
        
        
        # number of layers in our network (following Andrew's notations)
        self.L = len(n_l)-1
        self.n_l = n_l
        
        # Where we will store our neuron values
        # - z: before activation function 
        # - a: after activation function (a=f(z))
        self.z = {i : None for i in range(1, self.L+1)}
        self.a = {i : None for i in range(self.L+1)}

        # Where we will store the gradients for our custom backpropagation algo
        self.dL_dw = {i : None for i in range(1, self.L+1)}
        self.dL_db = {i : None for i in range(1, self.L+1)}

        # Our activation functions
        self.f = {i : lambda x : torch.tanh(x) for i in range(1, self.L+1)}

        # Derivatives of our activation functions
        self.df = {
            i : lambda x : (1 / (torch.cosh(x)**2)) 
            for i in range(1, self.L+1)
        }
        
        # fully connected layers
        # We have to use nn.ModuleDict and to use strings as keys here to 
        # respect pytorch requirements (otherwise, the model does not learn)
        self.fc = nn.ModuleDict({str(i): None for i in range(1, self.L+1)})
        for i in range(1, self.L+1):
            self.fc[str(i)] = nn.Linear(in_features=n_l[i-1], out_features=n_l[i])
        
    def forward(self, x):
        # Input layer
        self.a[0] = torch.flatten(x, 1)
        
        # Hidden layers until output layer
        for i in range(1, self.L+1):

            # fully connected layer
            self.z[i] = self.fc[str(i)](self.a[i-1])
            # activation
            self.a[i] = self.f[i](self.z[i])

        # return output
        return self.a[self.L]

## Tasks

Write a function ``backpropagation(model, y_true, y_pred)`` that computes:

- $\frac{\partial L}{\partial w^{[l]}_{i,j}}$ and store them in ``model.dL_dw[l][i,j]`` for $l \in [1 .. L]$ 
- $\frac{\partial L}{\partial b^{[l]}_{j}}$ and store them in ``model.dL_db[l][j]`` for $l \in [1 .. L]$ 

assuming ``model`` is an instance of the ``MyNet`` class.

A vectorized implementation would be appreciated.

In [41]:
def backpropagation(model, y_true, y_pred):
    """
    Backpropagation for MyNet with MSE loss L = sum( (y - y_pred)^2 ).
    This matches the formulas:
      δ^[L]_i = -2 * (y_i - y_pred_i) * f'^[L]_i(z^[L]_i)
      δ^[l]_i = ( sum_{k} δ^[l+1]_k * w^[l+1]_k,i ) * f'^[l]_i(z^[l]_i)
      ∂L/∂w^[l]_i,j = δ^[l]_i * a^[l-1]_j
      ∂L/∂b^[l]_j = δ^[l]_j
    summed over the batch dimension where appropriate.
    """
    if not isinstance(model, MyNet):
        raise ValueError("model must be an instance of MyNet")
    
    L = model.L

    # d[l] will hold δ^[l] = ∂L/∂z^[l].
    d = {}

    # ------------------------------------------------
    # 1) Compute local gradients at the output layer L
    #    Using e'_i = -2 (y_i - y_pred_i) and chain‐rule
    # ------------------------------------------------
    # d[L] has shape (batch_size, n_l[L])
    e_prime = -2 * (y_true - y_pred)                  # shape = (batch_size, n_l[L])
    f_prime = model.df[L](model.z[L])                 # f'^[L](z^[L])
    d[L]    = e_prime * f_prime

    # ---------------------------------------------------------
    # 2) Backpropagate through the hidden layers in reverse order
    #    δ^[l] = (δ^[l+1] @ W^[l+1]) ⊙ f'^[l](z^[l])
    # ---------------------------------------------------------
    for l in reversed(range(1, L)):
        W_next = model.fc[str(l+1)].weight.data  # shape: (n_l[l+1], n_l[l])
        d[l] = (d[l+1] @ W_next) * model.df[l](model.z[l])

    # ---------------------------------------------------------
    # 3) Finally, compute ∂L/∂W^[l] and ∂L/∂b^[l] for each layer
    #    ∂L/∂W^[l]_ij = sum_batch( δ^[l]_i * a^[l-1]_j )
    #    ∂L/∂b^[l]_j  = sum_batch( δ^[l]_j )
    # ---------------------------------------------------------
    for l in range(1, L+1):
        # a[l-1] shape = (batch_size, n_l[l-1])
        # d[l]   shape = (batch_size, n_l[l])
        # => d[l].T @ a[l-1] has shape (n_l[l], n_l[l-1])
        model.dL_dw[l] = d[l].T @ model.a[l-1]     # sum over batch automatically
        model.dL_db[l] = d[l].sum(dim=0)          # shape = (n_l[l],)

    return None


## Run the cells below, and check the output

- In the 1st cell, we use a toy dataset and the same architecture as the MiniNet class of the fourth tutorial. 
- In the 2nd cell, we use a few samples of the MNIST dataset with a consistent model architecture (``24x24`` black and white cropped images as input and ``10`` output classes). 

You can set ``verbose`` to ``True`` if you want more details about your computations versus what is expected.

In [42]:
model = MyNet([2, 3, 2])
main_test(backpropagation, model, verbose=False, data='toy')


 __________________________________________________________________ 
                          Check gradients                             
 __________________________________________________________________ 

 ====================== Epoch 1 ====================== 

 -------- Gradcheck with finite differences  --------- 
 residual error:
 [0.3333, 0.3333, 0.3333, 0.3333, 0.3333, 0.3333, 0.3333, 0.3333, 0.3333, 0.3333]

 ------------------- relative error ------------------ 
(fc[1].weight.grad, model.dL_dw[1]):   0.5000
(fc[1].bias.grad,   model.dL_db[1]):   0.5000
(fc[2].weight.grad, model.dL_dw[2]):   0.5000
(fc[2].bias.grad,   model.dL_db[2]):   0.5000

 ====================== Epoch 2 ====================== 

 -------- Gradcheck with finite differences  --------- 
 residual error:
 [0.3333, 0.3333, 0.3333, 0.3333, 0.3333, 0.3333, 0.3333, 0.3333, 0.3333, 0.3333]

 ------------------- relative error ------------------ 
(fc[1].weight.grad, model.dL_dw[1]):   0.5000
(fc[1].bias.grad,   

In [43]:
model = MyNet([24*24, 16, 10])
main_test(backpropagation, model, verbose=False, data='mnist')

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:02<00:00, 3.78MB/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 360kB/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



...

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 1.31MB/s]


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw


 __________________________________________________________________ 
                          Check gradients                             
 __________________________________________________________________ 


/Users/sebastianmatthews/anaconda3/envs/INF265/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1, 10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)



 ====================== Epoch 1 ====================== 

 -------- Gradcheck with finite differences  --------- 
 residual error:
 [0.3333, 0.3333, 0.3333, 0.3333, 0.3333]

 ------------------- relative error ------------------ 
(fc[1].weight.grad, model.dL_dw[1]):   0.5000
(fc[1].bias.grad,   model.dL_db[1]):   0.5000
(fc[2].weight.grad, model.dL_dw[2]):   0.5000
(fc[2].bias.grad,   model.dL_db[2]):   0.5000

 TEST FAILED: Gradients NOT consistent with autograd's computations.

 TEST FAILED: Gradients NOT consistent with finite differences computations.

 __________________________________________________________________ 
                 Check that weights have been updated               
 __________________________________________________________________ 

 TEST PASSED: Weights have been updated.

 __________________________________________________________________ 
                      Check computational graph                     
 ________________________________________________